# Why asyncio?

Asyncio, as a concurrency framework, allows us to do a *lot* at once. Importantly, it does so *very quickly*, beating threading in many common scenarions whilst also being easier to use.

## Networking speed:

In [ ]:
import requests
import aiohttp
import asyncio
from ipywidgets import widgets
from concurrent import futures

In [ ]:
r = requests.get('https://www.google.com/')

In [ ]:
async with aiohttp.ClientSession() as session:
    async with session.get('https://www.google.com/') as r:
        await r.read()

In [ ]:
session=requests.session()

for _ in range(20):
    session.get('https://www.google.com/')

In [ ]:
async with aiohttp.ClientSession() as session:

    async def get(url):
        async with session.get(url) as response:
            return await response.read()

    responses = await asyncio.gather(*[get('https://www.google.com/') for _ in range(20)])
    
    [r for r in responses]

In [ ]:
with futures.ThreadPoolExecutor(max_workers=2) as executor:
    responses = [executor.submit(lambda: requests.get("https://www.google.com/")) for _ in range(20)]

    [f.result().status_code for f in responses]

### Request speed, overall:
<img src="assets/requests performance.png" width="500" height="400">

As you can see, even in single-use cases aiohttp has an edge (probably because it is implemented in C, unlike requests) and the edge grows substantially when you want to do multiple tasks.

### asyncpg (postgres) preformance:
<img src="assets/asyncpg%20performance.png" width="500" height="400">

[Source](https://gistpreview.github.io/?b8eac294ac85da177ff82f784ff2cb60)

Postgres is another common tool, and the [`asyncpg`](https://github.com/MagicStack/asyncpg) is insanely fast at working with it- not to mention the other benefits of async programming.

# Great, what is it?

Asyncio is an async concurrency framework- running multiple tasks in paralell, in an overlapping way. To put it to a metaphor:
> Chess master Judit Polgár hosts a chess exhibition in which she plays multiple amateur players. She has two ways of conducting the exhibition: synchronously and asynchronously.
>
> Assumptions:
>
> 24 opponents
Judit makes each chess move in 5 seconds
Opponents each take 55 seconds to make a move
Games average 30 pair-moves (60 moves total)
Synchronous version: Judit plays one game at a time, never two at the same time, until the game is complete. Each game takes (55 + 5) * 30 == 1800 seconds, or 30 minutes. The entire exhibition takes 24 * 30 == 720 minutes, or 12 hours.
>
> Asynchronous version: Judit moves from table to table, making one move at each table. She leaves the table and lets the opponent make their next move during the wait time. One move on all 24 games takes Judit 24 * 5 == 120 seconds, or 2 minutes. The entire exhibition is now cut down to 120 * 30 == 3600 seconds, or just 1 hour.

[Source](https://www.youtube.com/watch?v=iG6fr81xHKA&t=269s)

To extend this metaphor, a truly parallel version would have a clone of Judit per table, which would theoretically drag the time to complete the tournament down to 30 minutes! However: true parallelism is exceptionally hard- in fact, Python *Doesn't actually have real threading.*

## Where does the performance come from? 
When doing IO, most of our time is spent waiting for a response. For example, when doing a GET request, building the request only takes a couple ms- but waiting for the response takes hundreds if not thousands of ms. While we're waiting: why not do something else? One of the things syncio aims to do is fill in the space while we wait.

## Threading isn't real
Python threading isn't "true" threading, due to the GIL- the global interpreter lock. 
The GIL prevents any two lines of Python code (strictly speaking, python bytecode) from executing at the same time- this is done due to the way Python stores references to objects in memory, and prevents some nasty race conditions and helps thread saftey. 

The downside of this is Python code cannot utilize the full ability of a processor- but it does make it faster in single-core preformance, generally. 

Non-Python code, such as a C code like much of the stdlib is implemented in and many third-party libaries like numpy, tensorflow, and pillow, have parts that do *not* trigger the GIL, and can run while the GIL is doing "other" things. Note that C code eventually has to return to Python code.

Additionally, spinning up threads and context-switches aren't free either.

### A note on multiprocessing
When multiprocessing, every subprocess has it's own GIL, and allows for true concurrency, at the cost of being insufferable to use, particularily on Windows. 

## or, in image form:
<img src="assets/concurrency blocks.png" width="500" height="400">

Any overlapping blocks represent C code running. Python bytecode will *never* overlap. Additionally, the thread scheduler is much more regular than the async event loop, so this image is a bit missleading.

# Great, how do we use it?

In [ ]:
async def func():
    await asyncio.sleep(5)
    return 'a'

`async def` indicates that this function is a coroutine (or, async generator). 

`await` does two things- it tells the async event loop to context switch to another task until the coroutine it's `await`ing has completed. When it does, get the return value and continue. 

`async with` is less common, but this is an asyncronous context manager- used to handle cleanup of something, usually. 

`async for` is for iterating over an asyncronous generator.

`return` does exactly as you'd think.

When we just call the coroutine function `func()` we see that it returns a coroutine:

In [ ]:
func() 

By `await`ing the returned coroutine, we actually schedule the coroutine to happen and run it- it then waits for a return value.

In [ ]:
await func()

You cannot use the `await`, `async with` and `async for` keywords outside of a coroutine function:

In [ ]:
def x():
    await asyncio.sleep(1)

### A note on jupyter:
Because we're in a jupyter notebook, we can use `await`, `async with` and `async for` as much as we please, because jupyter (or rather, `ipython`) is already running an async loop! In a normal python enviroment, you can't do this, but must instead have a start point as defined in [Entry points.](#Entry-points)

Several other common Python enviroments also create async loops such as Conda and Spyder- it can be useful to use [`nest_asyncio`](https://github.com/erdewit/nest_asyncio) in such situations.

### Definition: Coroutine, coroutine function
Like a regular function, but cooler. <br>
Coroutines are a generalized form of a subroutine, and can be entered and exited at multiple points. <br>
A coroutine function is a function that returns a coroutine, and may contain `await`, `async for` and `async with` keywords.<br>
**Key point: coroutine functions are defined with `async def`**<br>
Making a function a coroutine does not make the code inside it non-blocking (defined later).<br>

### Definition: awaitable

An `awaitable` is any object that can be `await`ed. These include, most commonly, `coroutines` and `tasks`.
Tasks are wrapped coroutines that are scheduled to happen immediately. More on these later.

### Definition: blocking

Blocking is any code that takes more than .1 seconds to return or yield back to the event loop. This prevents asyncronous programs from doing anything else at the same time- defined further in [blocking](#Blocking)

### A note on context managers
You'll very commonly see a lot of `async with` context managers in async code- moreso than you would see `with`s normally- this is in part because managing the start and stop points of an async function is kind of hard. If something needs a buildup and teardown, odds are it will use an async context manager.

## Entry points
All asyncronous programs need an "entry point" of sorts. Because we're in a jupyter notebook, we can `await` natively, as there is an async loop running in the background. However, in a normal python file we need to create this ourselves, such as with the following example.

```py
import asyncio

loop = asyncio.get_event_loop()

async def entry_point()
    await some_coroutine()
    
loop.run_until_complete(entry_point())
```

`asyncio.get_event_loop()` retreives the asyncio event loop if it exists, and if it doesn't, create it. 
`loop.run_until_complete()` runs the coroutine passed and blocks until it has completed. It cannot be used if the loop is already running. 

If you are using python 3.7+, you can replace this with [`asyncio.run()`](https://docs.python.org/3/library/asyncio-task.html#asyncio.run)

### A note on the event loop
The event loop is the scheduling queue asyncio runs on- when you run a coroutine, you schedule it to happen at some point in the future, and the asyncio event loop runs it at some point. Broadly speaking, you don't need to worry about it too much.

The only thing you need to really keep in mind is that a coroutine from one event loop can't be used in another event loop- such as when dealing with threads.

Additionally, there are multiple implementations of event loops- some event loops can cover multiple threads, and some that are [significantly faster, like uvloop](https://github.com/MagicStack/uvloop)

## Running multiple coroutines

So, asyncio allows us to run multiple things at the same time: but how do we do that?
### asyncio.create_task()
The first method is using [`asyncio.create_task()`](https://docs.python.org/3/library/asyncio-task.html#asyncio.create_task). This method takes in a coroutine, and immediately schedules it to happen, and immediately returns a [`task`](https://docs.python.org/3/library/asyncio-task.html#asyncio.Task) object. You can `await` the resulting `task` for a return value if you so desired- or you could call `task.cancel()` and stop the task from running. 

Tasks are commonly used to create a process running in the background- I want something to happen, but I don't care too much about the result of it. 

For example:

In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})
display(out)

# printing inside tasks is a little funny in jupyter, so we have to use this widget dance

async def my_task():
    await asyncio.sleep(3)
    with out:
        print('later')
    w.value += 'later\n'

asyncio.create_task(my_task())
with out:
    print('now')

Despite "my_task()" having been created and scheduled *before* we print `now`, `later` appears secondary. This is because we've scheduled it to happen "in the background" and continue with our program.

If we were to put a loop in our task, it could potentally run forever- useful for doing things repeatedly in the background.


We could then cancel this task by calling `task.cancel()`



In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})
display(out)

async def every_ten_seconds():
    while True:
        await asyncio.sleep(2)
        with out:
            print('something')
        
task = asyncio.create_task(every_ten_seconds())
print('hello')

In [ ]:
task.cancel()

#### task exceptions
It's worth noting that errors in a task will raise somewhat silently- until we get the return value. Consider the following code:

In [ ]:
async def raise_error():
    raise Exception()

task = asyncio.create_task(raise_error())

This will silently error in the background. (In jupyter, this will raise a warning) We can utilize `task.add_done_callback()` if we want it to visibly error, as shown here, or we could await the result of the task.

In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})
display(out)

task = asyncio.create_task(raise_error())

def exception_catching_callback(task):
    if task.exception():
        with out:
            task.print_stack()
        
task.add_done_callback(exception_catching_callback)

### asyncio.gather()
The second common thing we might want to do is run multiple coroutines at once *and* get the result. We can do that with [`asyncio.gather().`](https://docs.python.org/3/library/asyncio-task.html#running-tasks-concurrently) 

`gather` takes in n-many coroutines, runs them all at the same-ish time, waits for them all to complete, and returns all the values at once.

This was used prior in the examples of using web requests.

In [ ]:
async def my_coroutine(args):
    await asyncio.sleep(2)
    return args

results = await asyncio.gather(*[my_coroutine(x) for x in 'some words'])
results

We can use this to make an extremely silly time-based sorting function:

In [ ]:
unsorted = [5, 7, 3, 5, 1, 2, 3, 6, 9]
sorted_ = []

async def sorting_key(arg):
    await asyncio.sleep(arg)
    sorted_.append(arg)

await asyncio.gather(*[sorting_key(x) for x in unsorted])
sorted_

Note that the *order* of tasks inside gather is not guranteed, they will all happen "together".

### asyncio.wait()
[`asyncio.wait`](https://docs.python.org/3/library/asyncio-task.html#waiting-primitives) is similar to `asyncio.gather`, but it allows you to have some extra rules on when and what it returns by merit of it's `return_when` kwarg. The default behavior is to return when the first coroutine has completed (`asyncio.FIRST_COMPLETED`), but you can also return on the first exception (`asyncio.FIRST_EXCEPTION`) or when they have all completed- the same behavior as `asyncio.gather` (`asyncio.ALL_COMPLETED`).

`asyncio.wait` returns two things- the tasks that have completed, and the tasks that are still incomplete and have not yet returned- to get their result, you would have to check `task.result` or await them.

In the following example, we create a coroutine that will return the string that has been passed to it, and sleep for the time passed to it.

We then pass two of these coroutines to asyncio.wait, and check what's in `done` and `pending`

In [ ]:
async def task(time, string):
    await asyncio.sleep(time)
    return string

done, pending = await asyncio.wait((task(3, 'three seconds'), task(2, 'Two seconds')), return_when=asyncio.FIRST_COMPLETED)

for coro in done:
    print(coro.result())
    
print(pending)



## asyncio.wait_for()
[`asyncio.wait_for`](https://docs.python.org/3/library/asyncio-task.html#asyncio.wait_for) Allows you to schedule a timeout for an awaitable- so, say we have a network request, wait for user input, etc, and if it/they take too long, raise an exception or do something else.

If a timeout occurs, it raises `asyncio.TimeoutError` and cancels the task.

In the following example, we create a coroutine that takes 5 seconds to run, but use wait_for to set a timeout on it's runtime to 3 seconds, thus raising an error and canceling.

In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})
display(out)

async def takes_too_long():
    with out:
        print('starting to sleep!')
    await asyncio.sleep(5)
    with out:
        print('finished sleeping!')
        
await asyncio.wait_for(takes_too_long(), 3)


# Synchronization
Asyncio has 4 [Synchronization Primitives](https://docs.python.org/3/library/asyncio-sync.html). We will be covering 3- `Lock`, `Event` and `Semaphore`. 
### Locks
[Lock](https://docs.python.org/3/library/asyncio-sync.html#lock)s are the most basic of sync methods- they allow for exclusive access to a resource, to ensure that only one coroutine is doing *something* with *something*. This might be done with say, a file or a database.

Basic usage is exampled thusly:


In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})
display(out)

lock = asyncio.Lock()

async def do_db_op():
    async with lock:
        await asyncio.sleep(3)
        with out:
            print('Did DB op!')
        
await asyncio.gather(do_db_op(), do_db_op())

In this example, our imaginary database (implemented by sleeping) can only have one thing calling execute at once. So, we make a lock and have `do_db_op` accquire the lock before contiuning. One task we create will get the lock, do it's operation, and the second will wait it's turn until the first task has completed, and then do the db op

### Events
[Events](https://docs.python.org/3/library/asyncio-sync.html#event) are very similar to locks, but they allow *multiple* tasks access at once. For example, we might want to set up a database connection before we allow some coroutines to run, or we might wait for authentication to complete before continuing. Another example might be waiting for a cache to populate from a database, while still allowing some tasks to continue.

An event is defined as `asyncio.Event()` and has a handful of important methods. 

| Method | Explanation |
| :--- | :--- | 
| *coroutine* `wait()` | waits for the event to be `set`. A `set` event is "released" and will allow the code to continue. |
| `set()` | sets the event, which allows all tasks waiting for the event to continue. |
| `clear()` | "locks" the event, and will cause all code to wait to continue. |
| `is_set()` | returns the state of the `Event` |


In the following example, note that the two running instances of `my_task()` will not print `"hello <number>"` until we run the cell after it- which releases the event.

In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})
display(out)

event = asyncio.Event()

async def my_task(number):
    with out:
        print('waiting!', number)
    await event.wait()
    with out:
        print('hello!', number)

asyncio.create_task(my_task(1))
asyncio.create_task(my_task(2))

In [ ]:
event.set()

### Semaphores
[`Semaphores`](https://docs.python.org/3/library/asyncio-sync.html#semaphore) are a type of `lock` but they allow a specific number of things access to a resource. For example, if you have some API that allows up to 10 requests to be active at once, you could use this to bound it. A `sempahore` has a default value of 1- which makes it functionally identical to a lock by default. Only by raising the value can we use it more flexibily.

In the following example, we create 6 tasks using a similar method as before, but bind them to a semaphore instead of event that allows 2 things to work at once. Thus, tasks will be completed in 2-size chunks.

In [ ]:
import datetime

In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})
display(out)
sem = asyncio.Semaphore(2)

async def my_task(number):
    with out:
        print('waiting!', number)
    async with sem:
        await asyncio.sleep(2)
        with out:
            print('hello!', number, datetime.datetime.now())

await asyncio.gather(*[my_task(n) for n in range(6)])

# Queues
Finally we'll touch on [`queues`](https://docs.python.org/3/library/asyncio-queue.html#queue) a bit. If you're familiar with threading queues, these operate in much the same fashion. Put things in on one end, and things pop out on another end (by default- there are more exotic queues out there).

Queues are used to... queue things. You might use one to create a playlist of songs, for example. Other tasks can add to the queue, and you would have one task that operates on the queue.

A queue can have a limit as well- to cap the number of items. Depending on methods used, it may raise or wait when you try to add to it. 

Selected methods:

| function | Explanation |
| :-- | :-- |
| *coroutine* `get()`  | This method waits until an item is in the queue, and returns it |
| `get_nowait()` | This method immediately gets an item, and returns it. If there is nothing, it raises |
| *coroutine* `join()` | This coroutine waits until the queue has had all tasks completed |
| *coroutine* `put()`  | This coroutine adds an item to the queue, and if the queue is full, waits until it can |
| `put_nowait()`       | This method adds an item to a queue without waiting. It will raise if the queue is full. | 
| `task_done()`        | This method should be called after you are finished with whatever you received with `get()` |
| `qsize()`            | Returns the exact number of items in the queue. |

Notice how some of these methods aren't coroutines- this forms a convenient way to interface between syncronous and asyncronous sections of your code.

In the following example, we define a queue and a worker task. The worker task will wait for something to appear in the queue, and when it has, it will "work" on it- in this case, sleeping. 

The cells following it create buttons to add to the queue- note how `put_nowait()` is not a coroutine, and does not need to be awaited. Observe the different behaviors when the queue's max size is reached between the two options. Also note that because `put_nowait()` is a sync function, it's very easy to add as a callback to our button- while the coroutine `put` was more trouble than it was worth at the time of writing.

In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})
display(out)
queue = asyncio.Queue(maxsize=10)

async def sleepy_worker():
    while True:
        sleep_for = await queue.get()
        
        await asyncio.sleep(sleep_for)
        
        with out:
            print('Slept for', sleep_for)
        
worker_task = asyncio.create_task(sleepy_worker())

In [ ]:
await queue.put(3)

In [ ]:
button = widgets.Button(description='Click me')
button.on_click(lambda _ : queue.put_nowait(2))
display(button)

In [ ]:
worker_task.cancel()
# As this task works "forever", we should close it!

## Producers and consumers
`producers` and `consumers` are a common practice in both async and threaded applications- and we already created one! The sleepy worker in the above example would be a "consumer"- and our buttons would be a producer.

We can make a more formal one by creating a distinct produce function and multiple workers- we'll also use `queue.join()` and `queue.task_done()` to wait for the queue to empty before continuing. Further, we'll assign each worker a "worker ID" so we can better track who's working on what.

The following cell defines a queue, a worker, and creates 3 worker tasks.

In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})
display(out)
queue = asyncio.Queue(maxsize=10)

async def sleepy_worker(id_):
    while True:
        sleep_for = await queue.get()
        
        await asyncio.sleep(sleep_for)
        
        with out:
            print(f'Worker {id_} slept for', sleep_for)
        queue.task_done()
            
worker_tasks = [asyncio.create_task(sleepy_worker(n)) for n in range(3)]

We then produce 10 units of work, and can watch the worker tasks work- we only created 3 workers, but we have 10 units of work, so work will be completed in chunks, and not completely concurrent.

Further, we wait for the queue's tasks to be completed using `queue.join`, detailed in the next section. After we join, we cancel the worker tasks as cleanup.

In [ ]:
producers = [asyncio.create_task(queue.put(n)) for n in range(10)]

await asyncio.sleep(1)

print('waiting for workers to finish!')
await queue.join()
print('workers complete!')
for worker in worker_tasks:
    worker.cancel()

### Queue.join and Queue.task_done
To detail `queue.join()` and `queue.task_done()` when you add an item to a queue, it increments an internal counter of how many tasks there are to do- and calling `task_done()` decrements that counter. For example, if we wanted to queue up 100 things and when we were done exit, if we were to just check `qsize()` we could get a state where the queue had no more items in it, but some of those items hadn't been processed by workers- and if we just relied on size, we would exit before all the work had been done.

To avoid doing so, we can use `queue.join()` which waits for the task counter to be zero, and then continues.

The following code uses the private method `_unfinished_tasks` to demonstrate this- you shouldn't access this attribute in normal uses, but we'll use it to showcase the difference between the size of a queue and how many tasks have been done. Notice how the queue is empty, but we still have tasks remaining, and that `queue.join()` holds until the task has been completed by pressing the button in the next cell.

In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})
display(out)

queue = asyncio.Queue()
queue.put_nowait('a')
queue.get_nowait()
with out:
    print("queue size:", queue.qsize(), "number of tasks remaining:", queue._unfinished_tasks)

async def waiter_task():
    await queue.join()

    with out:
        print("queue size:", queue.qsize(), "number of tasks remaining:", queue._unfinished_tasks)

asyncio.create_task(waiter_task())

In [ ]:
button = widgets.Button(description='Release the task')
button.on_click(lambda _ : queue.task_done())
display(button)

# Threads

Full usage of threads and asyncio falls a bit outside of the scope of this- for more detailed information, you can read [this section](https://docs.python.org/3/library/asyncio-dev.html#concurrency-and-multithreading) of the documentation. 

However, sometimes we need to call blocking code, or just thread something and get the result. Asyncio provides a very tidy interface to do this in the form of [`run_in_executor()`.](https://docs.python.org/3/library/asyncio-eventloop.html#asyncio.loop.run_in_executor) 

An executor, broadly speaking, is a class that runs code elsewhere- there are two included with asyncio, `ThreadPoolExecutor` and `ProcessPoolExecutor` which run a function in a thread and a subprocess respectively. The standard rules about threading and multiprocessing still apply.

The first argument passed is the Executor, which is the default `ThreadPoolExecutor` executor if `None` is passed.
### Thread saftey
The vast majority of async-related methods and objects are *not threadsafe*.

Basic example:

In [ ]:
import time
out = widgets.Output(layout={'border': '1px solid black'})
display(out)

def blocking_task(time_):
    time.sleep(time_)
    with out:
        print('hello!')
    
async def non_blocking_task():
    await asyncio.sleep(1)
    with out:
        print('aloha!')

loop = asyncio.get_event_loop()
asyncio.create_task(non_blocking_task())
await loop.run_in_executor(None, blocking_task, 3)

Notice how the non-blocking task was still able to run- this is detailed more in the next section.

If you needed to pass kwargs to the function being executed, you can utilize [`functools.partial`](https://docs.python.org/3/library/functools.html#functools.partial) like such:

In [ ]:
import functools
def blocking_task(time_, *, word):
    time.sleep(time_)
    print(word)

partial = functools.partial(blocking_task, 3, word='hello!')
    
await loop.run_in_executor(None, partial)

# Blocking
Blocking is the big bad nasty you can get into with async programming. 

**Any code segment that does not return within 100ms or yield back to the event loop is *blocking*** (realistically, you want to yield more often)

What does this mean? Why does it matter? Asyncio, being a concurrency framework on a single thread relies on context switches to achieve concurrency- and it can only switch contexts on an await. If you *prevent* it from doing so, nothing else can work at that time. 

In the following example, we create two loops that every second post a message. We then sleep for ten seconds using the *blocking* time.sleep instead of the *non blocking* asyncio.sleep, and we can observe as the tasks are unable to accomplish it's work during the blocking timespan.

If you replace `time.sleep(10)` with `await asyncio.sleep(10)` the code will work as expected

In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})
display(out)

import datetime

async def print_over_time(word):
    for x in range(10):
        with out:
            print('hello', word, x)
        await asyncio.sleep(1)

asyncio.create_task(print_over_time('a'))
asyncio.create_task(print_over_time('b'))

await asyncio.sleep(3)
with out:
    print('starting to block!', datetime.datetime.now().strftime('%H:%M:%S'))
    time.sleep(10)
    print('finished blocking!', datetime.datetime.now().strftime('%H:%M:%S'))

Common blocking operations include, but are not limited to:
- `time.sleep()` 
- Heavy math- `sum(i * i for i in range(10 ** 7))` for example
- blocking IO libraries such as `requests`, `urllib`, `psycopg2`
- Obscured heavy math such as `PIL` 
- Loading extremely large files
- Loops that do not yield back at some point

Much of the time you alleviate blocking code- for example, if we need to do heavy C-lib'd math (like `PIL`, `numpy`, `tensorflow`), we can use `run_in_executor` to run it in another thread, and wait for a returned response. Other times, if it's *pure python* blocking code that cannot be otherwise amerliorated, you should use the `ProcessPoolExecutor` with `run_in_executor`

Other times, such as with a loop that does not yield back to the event loop, you can force a yield by dropping an `await asyncio.sleep(0)` which allows for a context switch.

**When avaliable, an asyncronous library should always be used over a syncronous one in an executor.** 

**It is imparative to avoid blocking in an asyncronous program!**

An example of a blocking sync loop that yields to the event loop would be as follows

In [ ]:
x = 0
while x<1000000: # make this larger if you want to see this really fail
    x+=1
    await asyncio.sleep(0)

# Debug Mode
Asyncio has a [debug mode](https://docs.python.org/3/library/asyncio-dev.html#debug-mode) that amung other things, warns you when things are blocking, when you failed to await a coroutine, or when you call something non-threadsafe across threads.

# Making web requests


## aiohttp
[aiohttp](https://docs.aiohttp.org/en/stable/) is the most mature and well-documented web libary for async enviroments- it's very similar to `requests` and mirrors it's API in many places. It, unlike requests, has webserver websocket modules included, and is implemented in C.

In terms of general usage, the primary difference you'll experience with it and requests is that you must explicitly create an [`aiohttp.Clientsession()`](https://docs.aiohttp.org/en/stable/client_advanced.html#client-session) before doing any requests- unlike `requests` which has `requests.get` there is no `aiohttp.get`.

Code roughly equivilant to `requests.get()` is as follows:

In [ ]:
async with aiohttp.ClientSession() as session:
    async with session.get('http://example.org') as response:
        content = await response.read()

This works great for single calls, but if we're doing many calls it's a good idea to create a `ClientSession` and reuse it- as exampled below.

Note that we must close the clientsession when complete, or it will throw a warning. Additionally, if a clientsession is created *outside* of a coroutine, it will also throw a warning. Bluntly, neither of these warnings matter too much as long as they happen only once in the lifespan of a program.

In [ ]:
session = aiohttp.ClientSession()

async with session.get('http://example.org') as response:
    pass
async with session.get('http://google.com') as response:
    pass

await session.close()

Additionally, when you make a request with aiohttp, only the response headers are initially retreived. Only when you `await` the various content-reading methods will it begin to load content. 

### Further readings
Aiohttp cannot be totally covered in this document, but it's rather extensive documentation for the client can be found [here,](https://docs.aiohttp.org/en/stable/client.html#client) as well as the documentation for it's other modules such as the webserver and websocket client.

## httpx
[httpx](https://www.python-httpx.org/) is a still fairly immature python module, and I would not mention it barring the fact that it supports `http/2`, unlike aiohttp which does not.

# Practical Example

todo

# Vestigial example

The following code snippet creates an API server and requests something from it- I decided not to flesh this out further, but it's interesting so I'm leaving it here.

In [ ]:
from aiohttp import web

app = web.Application()
routes = web.RouteTableDef()

@routes.get('/')
async def root_get_handler(requests):
    return web.json_response({'message':'hello'})

app.add_routes(routes)

web_task = asyncio.create_task(web._run_app(app)) 
# normally you use web.run_app but we need a non-blocking version

def shutdown_server(button):
    web_task.cancel()
    button.description='Server stopped'
    button.disabled=True
    
    
close_button = widgets.Button(description='Stop Server')
close_button.on_click(lambda _:shutdown_server(close_button))
display(close_button)

In [ ]:
async with aiohttp.ClientSession() as session:
    async with session.get('http://localhost:8080/') as response:
        content = await response.read()
        print(content)